In [1]:
import keras 
from keras import optimizers
from keras import losses
from keras.models import Sequential,Model
from keras.layers import Dense,Input,Dropout,Embedding,LSTM
from keras.optimizers import RMSprop,Adam,Nadam
from keras.preprocessing import sequence 
from keras.callbacks import TensorBoard,CSVLogger
from keras.utils import plot_model
seed = 32

import sklearn
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
##from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.utils import shuffle

import pickle 

import seaborn as sns
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import tensorflow
import time 
import sys

Using TensorFlow backend.


# Load Data

In [2]:
df = pd.read_csv('kelibia_data.csv')
df = df.set_index(df['DATE']).drop(['DATE'], axis = 1)

In [3]:
df

,Temp_min,Temp_max,PLV,EVA,APP
DATE,,,,,
1986-08-02,24.5,29.4,0.0,3.3,0.000
1986-09-02,20.5,25.5,24.5,3.2,0.000
1986-09-10,21.3,28.2,0.8,2.9,0.000
1986-09-21,24.0,28.5,3.5,5.6,0.000
1986-09-25,19.3,26.6,0.5,2.5,0.000
...,...,...,...,...,...
2007-11-17,8.6,13.5,19.6,2.0,3.588
2007-11-24,14.8,21.7,0.2,1.4,3.588
2007-11-27,16.0,20.3,0.4,1.3,3.588


In [4]:
def transformer_df(df):
    dic={}
    for c in df.columns :
        if (df[c].dtype =="object"):
            encoder = LabelEncoder()
            encoder.fit(df[c])
            df[c]=encoder.transform(df[c])
            dic[c]=encoder
            print('ok')
    with open("./models/LabelEncoders_dic.pickle","wb") as f:
        pickle.dump(dic,f)
    return df

In [5]:
def shuffle_dataframe(df):
    df = shuffle(df)
    return df

In [6]:
def normaliser_all_columns(df):
    diction={}
    
    for c in df.columns :
        scaler=MinMaxScaler(feature_range=(0,1)).fit(df[c].values.reshape(-1,1))
        diction[c]=scaler        
        df[c]=scaler.transform(df[c].values.reshape(-1,1))
    with open("./models/MinMaxScalers_dic.pickle","wb") as f:
        pickle.dump(diction,f)
    return df

In [7]:
df=transformer_df(df)
df=normaliser_all_columns(df)
df=shuffle_dataframe(df)

In [8]:
from sklearn.model_selection import train_test_split
def split_data_into_train_test_sets(ts_features_targets):
    X = ts_features_targets.drop('APP', axis=1)
    y = ts_features_targets[['APP']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=seed, shuffle=True)
    return X_train, X_test, y_train, y_test

In [9]:
X = df.drop('APP', axis=1)
y = df[['APP']]
X_train, X_test, y_train, y_test = split_data_into_train_test_sets(df)

In [10]:
# scaling and transform data
from sklearn.preprocessing import PowerTransformer, StandardScaler
def data_scaling_transform(X_train= None, X_test= None, y_train= None, y_test= None, X= None, y= None, splited_data = True):
    ### transform input variables
    if (splited_data == True):
        scaler = StandardScaler().fit(X_train)
        X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
        ### transform target variables
        transformer = PowerTransformer().fit(y_train)
        y_train = transformer.transform(y_train)
        y_test = transformer.transform(y_test)     
        return X_train, X_test, y_train, y_test
    else:
        scaler = StandardScaler().fit(X)
        X = pd.DataFrame(scaler.transform(X), columns=X.columns, index=X.index)
        transformer = PowerTransformer().fit(y)
        y = transformer.transform(y)
        return X, y

In [11]:
X,y = data_scaling_transform(None, None, None, None, X, y, False)
X_train, X_test, y_train, y_test = data_scaling_transform(X_train, X_test, y_train, y_test, True)

In [12]:
def create_ae(input_dim):
    
    dae_input=Input(shape=(input_dim,)) #Define the input layer
    
    encoded= Dense(units=input_dim,activation="tanh")(dae_input) 
    encoded= Dense(units=int(input_dim),activation="tanh")(encoded) 
    encoded= Dense(units=int(input_dim),activation="tanh")(encoded)
    encoded= Dense(units=int(input_dim/6),activation="tanh")(encoded)
    encoded= Dense(units=3,activation="tanh")(encoded)
    encoded= Dense(units=int(input_dim/6),activation="tanh")(encoded)
    decoded= Dense(units=int(input_dim/4),activation="tanh")(encoded)
    decoded= Dense(units=int(input_dim/2),activation="tanh")(decoded)
    decoded= Dense(units=input_dim,activation="tanh")(decoded)
    decoded= Dense(units=input_dim,activation="linear",name='decoded')(decoded)
    
    model = Model(inputs=dae_input, outputs=decoded)
    model.compile(optimizer=RMSprop(),loss="mean_squared_error",metrics=["mae"])
    model.summary()
    plot_model(model,to_file='ae.png',show_shapes=True)
    return model

In [13]:
'''from keras import Model
from keras.layers import Input, Dense
from keras.optimizers import RMSprop

def create_ae(input_dim, latent_dim):
    
    input_layer=Input(shape=(input_dim,)) #Define the input layer
    
    encoded= Dense(units=input_dim,activation="tanh")(input_layer) 
    encoded= Dense(units=3,activation="tanh")(encoded) 
    
    laten_vector= Dense(units=latent_dim,activation="tanh")(encoded)

    decoded= Dense(units=3,activation="tanh")(laten_vector)
    decoded= Dense(units=input_dim,activation="tanh")(decoded)
    
    output_layer= Dense(units=input_dim,activation="linear",name='decoded')(decoded)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=RMSprop(),loss="mean_squared_error",metrics=["mae"])
    model.summary()
    #plot_model(model,to_file='ae.png',show_shapes=True)
    return model'''

'from keras import Model\nfrom keras.layers import Input, Dense\nfrom keras.optimizers import RMSprop\n\ndef create_ae(input_dim, latent_dim):\n    \n    input_layer=Input(shape=(input_dim,)) #Define the input layer\n    \n    encoded= Dense(units=input_dim,activation="tanh")(input_layer) \n    encoded= Dense(units=3,activation="tanh")(encoded) \n    \n    laten_vector= Dense(units=latent_dim,activation="tanh")(encoded)\n\n    decoded= Dense(units=3,activation="tanh")(laten_vector)\n    decoded= Dense(units=input_dim,activation="tanh")(decoded)\n    \n    output_layer= Dense(units=input_dim,activation="linear",name=\'decoded\')(decoded)\n    \n    model = Model(inputs=input_layer, outputs=output_layer)\n    model.compile(optimizer=RMSprop(),loss="mean_squared_error",metrics=["mae"])\n    model.summary()\n    #plot_model(model,to_file=\'ae.png\',show_shapes=True)\n    return model'

In [16]:
autoecoder = create_ae(X_train.shape[1])#, 2)

AssertionError: 

In [15]:
batchsize=100
epoch=10
start_time = time.time() 
autoecoder.fit(X_train,X_train,
              batch_size=batchsize,
              epochs=epoch,
              verbose=1,
              shuffle=True,
              validation_data=(X_test,X_test),
              callbacks=[TensorBoard(log_dir="./logs/autoencoder")])
print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'autoecoder' is not defined

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {"./logs/autoencoder/"}

### View the avrage length distribution of all data

In [ ]:
#normal_data=df.drop("APP",axis=1)
y_pred=autoecoder.predict(X)
y_dist=np.linalg.norm(X-y_pred,axis=-1)

plt.figure(figsize=(15,10))
plt.hist(y_dist,bins=200)
plt.xlabel("Normal Data Average Loss Length")
plt.ylabel("Number of data points")
plt.show()

In [ ]:
'''
y_pred=autoecoder.predict(X_test) # predict new data 
y_dist=np.linalg.norm(X_test-y_pred,axis=-1) # calculate distance between real  and predicted data
plt.figure(figsize=(15,10))
plt.hist(y_dist,bins=200)
plt.xlabel("Anomaly DataAverage Loss Length")
plt.ylabel("Number of data points")
plt.show()
'''

In [ ]:
def prediction(xtest, threshold_anomalies, percent_of_anoamlies):
    start_time = time.time() 
    y_pred=autoecoder.predict(xtest) 
    var_inter = xtest-y_pred
    y_dist=np.linalg.norm(var_inter,axis=-1) 
    result = []
    for index, elem in enumerate(y_dist):
        if elem >threshold_anomalies:
            values_more_then_the_limit = 0
            l= len(var_inter.columns)
            for item in var_inter.iloc[index]:
                if item > percent_of_anoamlies:
                    values_more_then_the_limit+=1
            if(values_more_then_the_limit/l)>=percent_of_anoamlies:
                result.append(('Element number {0} is a rare event'.format(index), str(round(values_more_then_the_limit/l,2))))
            else:
                result.append(('Element number {0} is an anomaly'.format(index), str(round(values_more_then_the_limit/l,2))))
    return result

In [ ]:
#y_pred=prediction(X_test, 5, 0.7)

In [ ]:
def predictions_with_statistical_data(xtest, threshold_anomalies, percent_of_anoamlies):
    cleaned_df = pd.DataFrame()
    y_pred=autoecoder.predict(xtest) 
    var_inter = xtest-y_pred
    y_dist=np.linalg.norm(var_inter,axis=-1) 
    string_result = []
    values_result = []
    for index, elem in enumerate(y_dist):
        features = {'Temp_min':0, 'Temp_max':0, 'PLV':0, 'EVA':0, "index":index}
        if elem >threshold_anomalies:
            values_more_then_the_limit = 0
            l= len(var_inter.columns)
            for idx, item in enumerate(var_inter.iloc[index]):
                if item > percent_of_anoamlies:
                    #values_more_then_the_limit+=1
                    if(idx in range(0,7)):
                        features['Temp_min'] = 1
                    if(idx in range(7,14)):
                        features['Temp_max'] = 1
                    if(idx in range(14,21)):
                        features['PLV'] = 1
                    if(idx in range(21,28)):
                        features['EVA'] = 1
            values_result.append(features)
            if features['Temp_min']==1 and features['Temp_max']==1 and features['PLV']==1 and features['EVA']==1:
                string_result.append('Element number {0} is a rare event'.format(features['index']))
            else:
                string_result.append('Element number {0} is an anomaly'.format(features['index']))
        else:
            cleaned_df = cleaned_df.append(var_inter.iloc[index])
                                                                               
    '''        
    if(values_more_then_the_limit/l)>=percent_of_anoamlies:
        result.append(('Element number {0} is a rare event'.format(index), str(round(values_more_then_the_limit/l,2))))
    else:
        result.append(('Element number {0} is an anomaly'.format(index), str(round(values_more_then_the_limit/l,2))))
    '''
    return values_result, string_result, cleaned_df

In [ ]:
#values_result, string_result, cleaned_df=predictions_with_statistical_data(X, 5, 0.7)

In [ ]:
def predictions_with_original_data(X, threshold_anomalies, max_gap_between_real_predicted_data, percent_of_anoamlies):
    cleaned_df = pd.DataFrame()
    y_pred=autoecoder.predict(X) 
    var_inter = X-y_pred
    y_dist=np.linalg.norm(var_inter,axis=-1) 
    string_result = []
    values_result = []
    for index, elem in enumerate(y_dist):
        features = {'Temp_min':0, 'Temp_max':0, 'PLV':0, 'EVA':0, "index":index}
        if elem >threshold_anomalies:
            values_more_then_the_limit = 0
            #l= len(var_inter.columns)
            for idx, item in enumerate(var_inter.iloc[index]):
                if (item > max_gap_between_real_predicted_data or item < -max_gap_between_real_predicted_data):
                    #values_more_then_the_limit+=1
                    if idx == 0 :
                        features['Temp_min'] = 1
                    if idx == 1 :
                        features['Temp_max'] = 1
                    if idx == 2 :
                        features['PLV'] = 1
                    if idx ==3:
                        features['EVA'] = 1
                    values_more_then_the_limit+= 1 
            if(values_more_then_the_limit/len(var_inter.iloc[index])> percent_of_anoamlies):
                print('this is a rare event')
            values_result.append(features)
            if features['Temp_min']==1 and features['Temp_max']==1 and features['PLV']==1 and features['EVA']==1:
                string_result.append('Element number {0} is a rare event'.format(features['index']))
            else:
                string_result.append('Element number {0} is an anomaly'.format(features['index']))
                
            cleaned_df[X.iloc[index].name] = [np.nan, np.nan, np.nan, np.nan]
        else:
            cleaned_df[X.iloc[index].name] = X.iloc[index]
                                                                               
    '''        
    if(values_more_then_the_limit/l)>=percent_of_anoamlies:
        result.append(('Element number {0} is a rare event'.format(index), str(round(values_more_then_the_limit/l,2))))
    else:
        result.append(('Element number {0} is an anomaly'.format(index), str(round(values_more_then_the_limit/l,2))))
    '''
    return values_result, string_result, cleaned_df.T

In [ ]:
threshold_anomalies = 5
max_gap_between_real_predicted_data = 2.5 
percent_of_anoamlies = 0.7
values_result, string_result, cleaned_df=predictions_with_original_data(X, threshold_anomalies,
                                                                        max_gap_between_real_predicted_data,
                                                                        percent_of_anoamlies)

In [ ]:
autoecoder.save("./models/ae.h5")

In [ ]:
cleaned_df = pd.merge(cleaned_df, df[['APP']], left_index=True, right_index=True)

In [17]:
cleaned_df.index.name = 'DATE'
#cleaned_df['DATE'] = cleaned_df.index.get_level_values('DATE') 
cleaned_df.to_excel('./cleaned_data.xlsx', index= True)

NameError: name 'cleaned_df' is not defined

In [18]:
df.to_excel('./cleaned_data.xlsx', index= True)

In [ ]:
/home/houssem/Téléchargements/RAPPORT DE TRAVAIL DU MOIS JUIN.pdf